v0 + 범주형 데이터 label encoder 대신에 one hot encoding으로 변경(ID, shipmanager 는 label encoder 사용)

In [1]:
import warnings
warnings.filterwarnings(action="ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bisect
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
train_df = pd.read_parquet("../data/train_v0.parquet")
test_df = pd.read_parquet("../data/test_v0.parquet")

In [3]:
train_df.head()

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,...,BDI_ADJ,PORT_SIZE,CI_HOUR,year,month,day,hour,minute,weekday,WITlt0
0,CN,EKP8,Bulk,0.369560,Z517571,3.0,1.882568,0.149641,2.0,1.0,...,0.080405,0.935023,3.048333,2020,10,15,4,3,3,0
1,CN,EUC8,Container,1.420706,U467618,3.0,0.036071,-0.404179,2.0,1.0,...,1.002482,0.880050,17.138611,2019,9,17,2,55,1,0
2,CN,NGG6,Container,2.301099,V378315,5.0,-1.100235,0.808875,2.0,1.0,...,-1.008252,1.033596,98.827500,2019,2,23,6,43,5,0
3,JP,TMR7,Cargo,-0.625047,B726632,1.0,2.592759,-0.977036,1.0,0.0,...,-0.241418,-0.963324,0.000000,2020,9,18,22,6,4,0
4,RU,NNC2,Container,-0.339843,D215135,3.0,-0.674120,-0.566234,1.0,1.0,...,-0.325196,-0.811344,96.030556,2022,8,13,12,57,5,0


In [4]:
train_df.columns

Index(['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'ID', 'BREADTH',
       'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH',
       'SHIPMANAGER', 'FLAG', 'ATA_LT', 'DUBAI', 'BRENT', 'WTI', 'BDI_ADJ',
       'PORT_SIZE', 'CI_HOUR', 'year', 'month', 'day', 'hour', 'minute',
       'weekday', 'WITlt0'],
      dtype='object')

In [5]:
assert sorted(train_df["ARI_CO"].unique()) == sorted(test_df["ARI_CO"].unique())

assert sorted(train_df["ARI_PO"].unique()) == sorted(test_df["ARI_PO"].unique())

assert sorted(train_df["SHIP_TYPE_CATEGORY"].unique()) == sorted(test_df["SHIP_TYPE_CATEGORY"].unique())

# assert sorted(train_df["ID"].unique()) == sorted(test_df["ID"].unique())

# assert sorted(train_df["SHIPMANAGER"].unique()) == sorted(test_df["SHIPMANAGER"].unique())

# assert sorted(train_df["FLAG"].unique()) == sorted(test_df["FLAG"].unique())

In [6]:
a = set(sorted(train_df["FLAG"].unique()))
print(len(a))
b = set(sorted(test_df["FLAG"].unique()))
print(len(b))
a = a | b
print(len(a))

123
119
126


In [7]:
a = set(sorted(train_df["ID"].unique()))
print(len(a))
b = set(sorted(test_df["ID"].unique()))
print(len(b))
a = a | b
print(len(a))

24763
23249
26702


In [3]:
# train, test 동일한 경우
train_df = pd.get_dummies(train_df, columns=["ARI_CO", "ARI_PO", "SHIP_TYPE_CATEGORY"], drop_first=True)
test_df = pd.get_dummies(test_df, columns=["ARI_CO", "ARI_PO", "SHIP_TYPE_CATEGORY"], drop_first=True)

In [4]:
# train, test 다른 경우
train_dummies = pd.get_dummies(train_df, columns=["FLAG"])
test_dummies = pd.get_dummies(test_df, columns=["FLAG"])

In [5]:
train_df, test_df = train_dummies.align(test_dummies, axis=1, fill_value=0)

In [6]:
assert sorted(list(train_df.columns)) == sorted(list(test_df.columns))

In [7]:
categorical_features = ['ID', 'SHIPMANAGER']
encoders = {}

for feature in categorical_features:
  le = LabelEncoder()
  train_df[feature] = le.fit_transform(train_df[feature].astype(str))
  le_classes_set = set(le.classes_)
  test_df[feature] = test_df[feature].map(lambda s: '-1' if s not in le_classes_set else s)
  le_classes = le.classes_.tolist()
  bisect.insort_left(le_classes, '-1')
  le.classes_ = np.array(le_classes)
  test_df[feature] = le.transform(test_df[feature].astype(str))
  encoders[feature] = le

In [9]:
test_df.drop(columns=["CI_HOUR"], inplace=True)

In [10]:
train_df.to_parquet("../data/train_v1.parquet")
test_df.to_parquet("../data/test_v1.parquet")